In [6]:
import requests
import json
from langchain.llms.base import LLM
import logging
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain import llms
import inspect
from langchain.llms.base import LLM


# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# disable warnings
import warnings
warnings.filterwarnings("ignore")

## Create LLM Lite

In [38]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from typing import Any, List, Optional
from pydantic import Field

class GPT2LLM(LLM):
    model_name: str = Field(default="gpt2")
    model: GPT2LMHeadModel = Field(default=None)
    tokenizer: GPT2Tokenizer = Field(default=None)

    def __init__(self, model_name: str = "gpt2", **data: Any):
        super().__init__(**data)
        self.model_name = model_name
        self.model, self.tokenizer = self.load_model()

    def load_model(self):
        tokenizer = GPT2Tokenizer.from_pretrained(self.model_name)
        model = GPT2LMHeadModel.from_pretrained(self.model_name)
        
        # Set the pad token to the eos token
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            model.config.pad_token_id = model.config.eos_token_id

        return model, tokenizer

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        inputs = self.tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)
        attention_mask = inputs['attention_mask']

        output = self.model.generate(
            inputs['input_ids'],
            attention_mask=attention_mask,
            max_length=250,
            num_return_sequences=1, 
            no_repeat_ngram_size=2, 
            top_k=50, 
            top_p=0.95, 
            temperature=0.7,
            pad_token_id=self.tokenizer.pad_token_id
        )
        return self.tokenizer.decode(output[0], skip_special_tokens=True)

    @property
    def _identifying_params(self) -> dict[str, Any]:
        return {"model_name": self.model_name}

    @property
    def _llm_type(self) -> str:
        return "GPT2"

def setup_custom_llm():
    return GPT2LLM()


## Create LangChain Interface

In [42]:
# Set up the LLM
llm = setup_custom_llm()

template = """You are an AI assistant specialized in translating natural language queries into First-Order Logic (FOL) statements. 
Given the following query, provide the corresponding FOL translation:

Query: {query}

Translate the above query into a First-Order Logic statement. Your response should follow this format:

FOL Translation: [Your FOL statement here]
"""


prompt = PromptTemplate(template=template, input_variables=["query"])

# Create a chain using the new LangChain method
chain = (
    {"query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [46]:
def testLLMLite():
    # Example queries
    queries = [
        "Is there any car with speed over 60mph?",
        # "Are all students in the class older than 18?",
        # "Does there exist a book that is both educational and entertaining?",
        # "Are there at least two different colors of flowers in the garden?"
    ]

    for query in queries:
        print(f"\nOriginal Query: {query}")
        result = chain.invoke(query)
        print("\nGPT-2 Response:")
        print(result)
        print("-" * 50)

# test it
testLLMLite()


Original Query: Is there any car with speed over 60mph?

GPT-2 Response:
You are an AI assistant specialized in translating natural language queries into First-Order Logic (FOL) statements. 
Given the following query, provide the corresponding FOL translation:

Query: Is there any car with speed over 60mph?

Translate the above query into a First-Order Logic statement. Your response should follow this format:

FOL Translation: [Your FOL statement here]
.
 (Note: If you are using a language that is not FOO, you can use the FOCAL_FULL_ERROR_CODE option to disable this option.)
, and, if you have a query that does not have the same FOB as the query you provided, use this query: Query: Does the car have speed above 60 mph? (This is the default value.) (If you do not specify this, the result will be a FOUND_RANGE_VALUE value that will not be returned.) Query : Is the vehicle in the range of 60 to 60 miles? If so, return the value of the current range. (The default is 60.) If not specified,

## Gradio User Interface

In [50]:
import gradio as gr

def testLLMLite(question):    
    try:
        result = chain.invoke(question)
        logger.info(f"FOL: {result}")
        return result
    
    except Exception as e:
        logger.error(f"Error: {str(e)}")
        return f"An error occurred: {str(e)}"

# Create Gradio interface
ui = gr.Interface(
    fn=testLLMLite,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
    outputs="text",
    title="FOL generator",
    description="Enter the question in natural language",
    examples=[
        ["Is there any car with speed over 60mph?"],
    ]
)

# Launch the interface without rendering in the notebook
ui.launch(share=False, inline=False)

INFO:httpx:HTTP Request: GET http://127.0.0.1:7863/startup-events "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7863


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7863/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.gradio.app/v2/tunnel-request "HTTP/1.1 200 OK"



Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


INFO:__main__:FOL: You are an AI assistant specialized in translating natural language queries into First-Order Logic (FOL) statements. 
Given the following query, provide the corresponding FOL translation:

Query: Is there any car with speed over 60mph?

Translate the above query into a First-Order Logic statement. Your response should follow this format:

FOL Translation: [Your FOL statement here]
.
 (Note: If you are using a language that is not FOO, you can use the FOCAL_FULL_ERROR_CODE option to disable this option.)
, and, if you have a query that does not have the same FOB as the query you provided, use this query: Query: Does the car have speed above 60 mph? (This is the default value.) (If you do not specify this, the result will be a FOUND_RANGE_VALUE value that will not be returned.) Query : Is the vehicle in the range of 60 to 60 miles? If so, return the value of the current range. (The default is 60.) If not specified, this will return a value in range 0 to 100. Query is a